In [2]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
전자결재 문서를 {문서ID: 압축된HTML} 딕셔너리로 변환
- Java 코드(AnyFiveNewCrawler_docBody)와 동일한 HTML 처리 로직 적용
"""
import json
import re
from pathlib import Path
from bs4 import BeautifulSoup

# 이미지 경로 프리픽스
IMG_PATH_PREFIX = "/PMS_SITE-U7OI43JLDSMO/approval/approval_2025_new_img"

# CSS 파일 경로 (Java와 동일)
CSS_FILE_PATH = "C:/Users/LEEJUHWAN/Downloads/2021-01-01~2025-10-31/html/resource/css/apr.doc.print.jstl.css"


def load_and_process_css(css_file_path):
    """
    CSS 파일 로드 및 가공 (Java의 loadAndProcessCss와 동일)
    """
    try:
        # EUC-KR 인코딩으로 읽기
        with open(css_file_path, 'r', encoding='euc-kr') as f:
            css = f.read()
    except FileNotFoundError:
        print(f"⚠️  CSS 파일을 찾을 수 없습니다: {css_file_path}")
        print("   JSON 내의 cssContent를 대신 사용합니다.")
        return None
    except Exception as e:
        print(f"⚠️  CSS 파일 읽기 실패: {e}")
        return None

    # 1. 커스텀 CSS 맨 앞에 추가
    custom_css = ".content{width:80% !important;}.content table{width:100% !important;}"

    # 2. html { } 규칙 전체 삭제
    css = re.sub(r'html\s*\{[^}]*\}', '', css)

    # 3. 선택자에서 html, body 제거
    css = re.sub(r'html\s*,\s*body\s*,\s*', '', css)
    css = re.sub(r'html\s*,\s*', '', css)
    css = re.sub(r'body\s*,\s*', '', css)

    # 4. .content_box 규칙 수정
    css = re.sub(
        r'\.content_box\s*\{[^}]*\}',
        '.content_box{border:1px #000000 solid;margin:5px 0 0 0;word-break:break-all;overflow:hidden;min-width:819px;}',
        css
    )

    # 5. ul, ol 규칙 수정
    css = re.sub(
        r'ul\s*,\s*ol\s*\{[^}]*\}',
        'ul,ol{list-style:none !important;padding-left:0 !important;margin-left:0 !important;}',
        css
    )

    # 6. CSS 압축 (minify)
    css = minify_css(css)

    # 7. 커스텀 CSS + 원본 CSS
    return custom_css + css


def minify_css(css):
    """CSS 압축 (Java의 minifyCss와 동일)"""
    # 주석 제거
    css = re.sub(r'/\*.*?\*/', '', css, flags=re.DOTALL)
    # 줄바꿈 제거
    css = css.replace('\n', '').replace('\r', '')
    # 연속 공백을 하나로
    css = re.sub(r'\s+', ' ', css)
    # { } : ; , 앞뒤 공백 제거
    css = re.sub(r'\s*\{\s*', '{', css)
    css = re.sub(r'\s*\}\s*', '}', css)
    css = re.sub(r'\s*:\s*', ':', css)
    css = re.sub(r'\s*;\s*', ';', css)
    css = re.sub(r'\s*,\s*', ',', css)
    return css.strip()


def minify_html(html):
    """HTML 압축 (Java의 minifyHtml과 동일)"""
    # 1. HTML 주석 제거
    html = re.sub(r'<!--.*?-->', '', html, flags=re.DOTALL)
    # 2. 태그 사이 공백 제거
    html = re.sub(r'>\s+<', '><', html)
    # 3. 연속 공백을 하나로
    html = re.sub(r'\s+', ' ', html)
    # 4. 줄바꿈 제거
    html = html.replace('\n', '').replace('\r', '')

    return html.strip()


def process_html(full_html, doc_id, css_content):
    """
    HTML 가공 (Java의 processHtml과 동일)
    - div.content 영역 추출
    - 불필요 섹션 삭제
    - 이미지 src 변환
    - div#middle로 감싸기 + style 태그 추가
    """
    # BeautifulSoup으로 파싱
    soup = BeautifulSoup(full_html, 'html.parser')

    # 1. div.content 영역 추출
    content_div = soup.select_one('div.content')
    if content_div is None:
        print(f"    ⚠️  div.content를 찾을 수 없습니다.")
        return ""

    # 2. 불필요 섹션 삭제

    # 2-1. 첨부파일/참조문서 영역 삭제 (id="attachFileArea")
    attach_file_area = content_div.select_one('#attachFileArea')
    if attach_file_area:
        attach_file_area.decompose()

    # 2-2. 결재의견 영역 삭제 (table04_box 중 caption이 "결재의견")
    table04_boxes = content_div.select('div.table04_box')
    for box in table04_boxes:
        caption = box.select_one('caption')
        if caption and '결재의견' in caption.get_text():
            box.decompose()

    # 2-3. 결재댓글 영역 삭제 (id="cmtArea")
    cmt_area = content_div.select_one('#cmtArea')
    if cmt_area:
        cmt_area.decompose()

    # 2-4. 조회자 행 삭제 (id="hitPwrListTr")
    hit_pwr_list_tr = content_div.select_one('#hitPwrListTr')
    if hit_pwr_list_tr:
        hit_pwr_list_tr.decompose()

    # 3. 이미지 src 변환
    img_tags = content_div.select('img')
    for idx, img in enumerate(img_tags):
        new_src = f"{IMG_PATH_PREFIX}/apr{doc_id}/{idx}.jpg"
        img['src'] = new_src

    # 4. div#middle로 감싸기 + style 태그 추가
    content_html = (
        f'<div id="middle" style="margin-top:45px;">'
        f'{str(content_div)}'
        f'</div>'
        f'<style>{css_content}</style>'
    )

    # 5. HTML 압축 (minify)
    result = minify_html(content_html)

    return result


def main():
    # 0. CSS 파일 로드 및 가공
    print("0단계: CSS 파일 로드 및 가공.")
    css_content = load_and_process_css(CSS_FILE_PATH)
    use_external_css = css_content is not None
    if use_external_css:
        print(f"  > CSS 파일 로드 완료. 길이: {len(css_content)}")
    else:
        print("  > 외부 CSS 파일 없음. 각 문서의 cssContent 사용.")

    # 1. JSON 파일 읽기
    input_file = Path("전자결재_완전한문서.json")

    if not input_file.exists():
        print(f"❌ 파일을 찾을 수 없습니다: {input_file}")
        return

    print(f"\n1단계: JSON 파일 읽는 중...")
    with open(input_file, "r", encoding="utf-8") as f:
        documents = json.load(f)

    print(f"  > {len(documents)}개 문서 로드 완료")

    # 2. 딕셔너리로 변환 (문서ID: 압축된HTML)
    print(f"\n2단계: HTML 변환 시작 (Java 로직 적용)...")
    result_dict = {}
    success_count = 0
    error_count = 0
    total_size = 0

    for i, doc in enumerate(documents, 1):
        try:
            doc_id = doc.get('문서ID')
            if not doc_id:
                print(f"⚠️  [{i}/{len(documents)}] 문서ID가 없습니다. 건너뜁니다.")
                error_count += 1
                continue

            title = doc.get('제목', f'문서_{i}')
            full_html = doc.get('fullHtml', '')

            if not full_html:
                print(f"⚠️  [{i}/{len(documents)}] fullHtml이 없습니다. 문서ID: {doc_id}")
                error_count += 1
                continue

            # CSS 결정: 외부 파일 우선, 없으면 문서 내 cssContent 사용
            if use_external_css:
                doc_css = css_content
            else:
                doc_css = doc.get('cssContent', '')
                doc_css = minify_css(doc_css) if doc_css else ''

            # HTML 가공 (Java processHtml과 동일)
            processed_html = process_html(full_html, doc_id, doc_css)

            if not processed_html:
                print(f"✗ [{i}/{len(documents)}] HTML 가공 실패. 문서ID: {doc_id}")
                error_count += 1
                continue

            # 딕셔너리에 추가
            result_dict[doc_id] = processed_html
            success_count += 1
            total_size += len(processed_html)

            print(f"✓ [{i}/{len(documents)}] 문서ID: {doc_id}")
            print(f"   제목: {title[:50]}{'...' if len(title) > 50 else ''}")
            print(f"   HTML 길이: {len(processed_html):,} bytes")

        except Exception as e:
            error_count += 1
            print(f"✗ [{i}/{len(documents)}] 에러: {str(e)}")
            continue

        # 10건마다 진행 상황 출력
        if i % 10 == 0:
            print(f"\n=== 진행 상황: {i}/{len(documents)} (성공: {success_count}, 실패: {error_count}) ===\n")

    # 3. JSON 파일로 저장
    output_file = Path("전자결재_문서ID별_HTML.json")

    print(f"\n3단계: JSON 파일 저장 중...")
    with open(output_file, "w", encoding="utf-8") as f:
        json.dump(result_dict, f, ensure_ascii=False, indent=2)

    # 4. 결과 요약
    print(f"\n✅ 완료!")
    print(f"   📊 통계:")
    print(f"      - 총 처리 대상: {len(documents)}개")
    print(f"      - 성공: {success_count}개")
    print(f"      - 실패: {error_count}개")
    print(f"      - 총 HTML 크기: {total_size:,} bytes")
    print(f"\n   📁 저장 위치: {output_file}")
    print(f"\n💡 사용 예시:")
    print(f"   import json")
    print(f"   with open('{output_file.name}', 'r', encoding='utf-8') as f:")
    print(f"       data = json.load(f)")
    print(f"   html = data['26426366']  # 문서ID로 바로 접근")


if __name__ == "__main__":
    main()

0단계: CSS 파일 로드 및 가공.
  > CSS 파일 로드 완료. 길이: 10111

1단계: JSON 파일 읽는 중...
  > 3개 문서 로드 완료

2단계: HTML 변환 시작 (Java 로직 적용)...
✓ [1/3] 문서ID: 2002390
   제목: 법인카드 사용내역 정산 보고_5월_편영수K
   HTML 길이: 97,532 bytes
✓ [2/3] 문서ID: 2008214
   제목: 2015년 1월 CARD 사용내역 정산 보고 2015년01월 법인카드 정산내역 품의(송준선...
   HTML 길이: 14,435 bytes
✓ [3/3] 문서ID: 2008497
   제목: [외주인력계약품의-대외]LGD SWP Project On-Time Visibility 확보...
   HTML 길이: 49,327 bytes

3단계: JSON 파일 저장 중...

✅ 완료!
   📊 통계:
      - 총 처리 대상: 3개
      - 성공: 3개
      - 실패: 0개
      - 총 HTML 크기: 161,294 bytes

   📁 저장 위치: 전자결재_문서ID별_HTML.json

💡 사용 예시:
   import json
   with open('전자결재_문서ID별_HTML.json', 'r', encoding='utf-8') as f:
       data = json.load(f)
   html = data['26426366']  # 문서ID로 바로 접근
